In [1]:
1+1

2

1. 코사인 유사도 계산

In [ ]:
from langchain_openai import OpenAIEmbeddings
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import os
from dotenv import load_dotenv

load_dotenv()
# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

# 비교할 텍스트들
text1 = "오늘 날씨가 정말 좋습니다."
text2 = "햇살이 따뜻하고 기분이 좋네요."
text3 = "컴퓨터 프로그래밍을 배우고 있습니다."

# 임베딩 생성
embedding1 = embeddings.embed_query(text1)
embedding2 = embeddings.embed_query(text2)
embedding3 = embeddings.embed_query(text3)

# 코사인 유사도 계산
similarity_1_2 = cosine_similarity([embedding1], [embedding2])[0][0]
similarity_1_3 = cosine_similarity([embedding1], [embedding3])[0][0]

print(f"텍스트1 vs 텍스트2 유사도: {similarity_1_2:.4f}")
print(f"텍스트1 vs 텍스트3 유사도: {similarity_1_3:.4f}")


#텍스트1 vs 텍스트2 유사도: 0.3452
#텍스트1 vs 텍스트3 유사도: 0.1175


텍스트1 vs 텍스트2 유사도: 0.3452
텍스트1 vs 텍스트3 유사도: 0.1175


2. 벡터 간 거리 계산

In [5]:
from scipy.spatial.distance import euclidean, cityblock

# 유클리드 거리 (낮을수록 유사)
euclidean_dist = euclidean(embedding1, embedding2)
print(f"유클리드 거리: {euclidean_dist:.4f}")

# 맨하탄 거리 (낮을수록 유사)
# 맨하탄 거리 (cityblock 사용)
manhattan_dist = cityblock(embedding1, embedding2)
print(f"맨하탄 거리: {manhattan_dist:.4f}")

유클리드 거리: 1.1443
맨하탄 거리: 28.8679


In [10]:
import numpy as np

# 리스트를 NumPy 배열로 변환
embedding1_array = np.array(embedding1)
embedding2_array = np.array(embedding2)

# 맨하탄 거리 계산
manhattan_dist = np.sum(np.abs(embedding1_array - embedding2_array))
print(f"맨하탄 거리: {manhattan_dist:.4f}")

# 직접 변환하면서 계산
manhattan_dist = np.sum(np.abs(np.array(embedding1) - np.array(embedding2)))
print(f"맨하탄 거리: {manhattan_dist:.4f}")

# 리스트 컴프리헨션 사용
manhattan_dist = sum(abs(a - b) for a, b in zip(embedding1, embedding2))
print(f"맨하탄 거리: {manhattan_dist:.4f}")

맨하탄 거리: 28.8679
맨하탄 거리: 28.8679
맨하탄 거리: 28.8679


다중 텍스트 유사도 매트릭스
여러 텍스트 동시 비교

In [11]:
#from langchain_openai import OpenAIEmbeddings
#import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 임베딩 모델 초기화
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 비교할 텍스트 리스트
texts = [
    "오늘 날씨가 정말 좋습니다.",
    "햇살이 따뜻하고 기분이 좋네요.",
    "비가 오고 있어서 우울합니다.",
    "파이썬 프로그래밍을 배우고 있습니다.",
    "머신러닝 공부가 재미있어요.",
]

# 모든 텍스트의 임베딩 생성
text_embeddings = embeddings.embed_documents(texts)

# 유사도 매트릭스 계산
similarity_matrix = cosine_similarity(text_embeddings)

# 결과를 DataFrame으로 시각화
df = pd.DataFrame(
    similarity_matrix,
    index=[f"텍스트{i+1}" for i in range(len(texts))],
    columns=[f"텍스트{i+1}" for i in range(len(texts))],
)

print("유사도 매트릭스:")
print(df.round(4))

유사도 매트릭스:
        텍스트1    텍스트2    텍스트3    텍스트4    텍스트5
텍스트1  1.0000  0.3452  0.3293  0.0828  0.2092
텍스트2  0.3452  1.0000  0.2684  0.1146  0.2278
텍스트3  0.3293  0.2684  1.0000  0.2536  0.1734
텍스트4  0.0828  0.1146  0.2536  1.0000  0.2478
텍스트5  0.2092  0.2278  0.1734  0.2478  1.0000


실용적인 유사도 검사 함수
1. 유사도 검사 클래스

In [ ]:
class SimilarityChecker:
    def __init__(self, model_name="text-embedding-3-small", threshold=0.8):
        self.embeddings = OpenAIEmbeddings(model=model_name, dimensions=1024)
        self.threshold = threshold

    def calculate_similarity(self, text1, text2):
        """두 텍스트 간의 코사인 유사도 계산"""
        embedding1 = self.embeddings.embed_query(text1)
        embedding2 = self.embeddings.embed_query(text2)

        similarity = cosine_similarity([embedding1], [embedding2])[0][0]
        return similarity

    def is_similar(self, text1, text2):
        """임계값 기준으로 유사 여부 판단"""
        similarity = self.calculate_similarity(text1, text2)
        return similarity >= self.threshold, similarity

    def find_most_similar(self, query_text, candidate_texts):
        """후보 텍스트 중 가장 유사한 것 찾기"""
        query_embedding = self.embeddings.embed_query(query_text)
        candidate_embeddings = self.embeddings.embed_documents(candidate_texts)

        similarities = cosine_similarity([query_embedding], candidate_embeddings)[0]

        # 가장 유사한 텍스트의 인덱스와 유사도
        best_idx = np.argmax(similarities)
        best_similarity = similarities[best_idx]

        return {
            "most_similar_text": candidate_texts[best_idx],
            "similarity_score": best_similarity,
            "index": best_idx,
            "all_similarities": similarities,
        }


# 사용 예시
checker = SimilarityChecker(threshold=0.7)

# 두 텍스트 유사도 검사
text_a = "파이썬으로 웹 개발을 하고 있습니다."
text_b = "Python을 사용해서 웹사이트를 만들고 있어요."

is_similar, score = checker.is_similar(text_a, text_b)
print(f"유사 여부: {is_similar}, 유사도: {score:.4f}")

# 가장 유사한 텍스트 찾기
query = "머신러닝 공부하는 방법"
candidates = [
    "AI 학습 방법론에 대해 알아보자",
    "요리 레시피를 찾고 있습니다",
    "딥러닝 튜토리얼 추천해주세요",
    "오늘 날씨가 좋네요",
]

result = checker.find_most_similar(query, candidates)
print(f"가장 유사한 텍스트: {result['most_similar_text']}")
print(f"유사도 점수: {result['similarity_score']:.4f}")

# 유사 여부: False, 유사도: 0.5388
# 가장 유사한 텍스트: AI 학습 방법론에 대해 알아보자
# 유사도 점수: 0.4301


유사 여부: False, 유사도: 0.5388
가장 유사한 텍스트: AI 학습 방법론에 대해 알아보자
유사도 점수: 0.4301


고급 유사도 검사 기법
1. 의미적 검색 구현

In [13]:
class SemanticSearch:
    def __init__(self, documents, model_name="text-embedding-3-small"):
        self.documents = documents
        self.embeddings = OpenAIEmbeddings(model=model_name, dimensions=1024)
        self.doc_embeddings = self.embeddings.embed_documents(documents)

    def search(self, query, top_k=3):
        """쿼리와 가장 유사한 문서들 반환"""
        query_embedding = self.embeddings.embed_query(query)

        # 모든 문서와의 유사도 계산
        similarities = cosine_similarity([query_embedding], self.doc_embeddings)[0]

        # 상위 k개 결과 선택
        top_indices = np.argsort(similarities)[::-1][:top_k]

        results = []
        for idx in top_indices:
            results.append(
                {
                    "document": self.documents[idx],
                    "similarity": similarities[idx],
                    "rank": len(results) + 1,
                }
            )

        return results


# 사용 예시
documents = [
    "파이썬은 프로그래밍 언어입니다.",
    "머신러닝은 인공지능의 한 분야입니다.",
    "오늘 날씨가 맑고 따뜻합니다.",
    "딥러닝은 신경망을 사용하는 기술입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.",
]

search_engine = SemanticSearch(documents)
results = search_engine.search("AI와 관련된 기술", top_k=3)

for result in results:
    print(f"순위 {result['rank']}: {result['document']}")
    print(f"유사도: {result['similarity']:.4f}\n")

순위 1: 머신러닝은 인공지능의 한 분야입니다.
유사도: 0.3628

순위 2: 자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.
유사도: 0.3310

순위 3: 파이썬은 프로그래밍 언어입니다.
유사도: 0.2773



In [14]:
def batch_similarity_check(texts, batch_size=100):
    """대량의 텍스트에 대한 효율적인 유사도 검사"""
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    # 배치 단위로 임베딩 생성
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = embeddings.embed_documents(batch)
        all_embeddings.extend(batch_embeddings)

    # 유사도 매트릭스 계산
    similarity_matrix = cosine_similarity(all_embeddings)

    return similarity_matrix

유사도 임계값 설정 가이드
임계값별 의미
0.9 이상: 거의 동일한 의미

0.8 - 0.9: 매우 유사한 의미

0.7 - 0.8: 관련성이 높음

0.6 - 0.7: 어느 정도 관련성 있음

0.5 이하: 관련성이 낮음

도메인별 권장 임계값
중복 검사: 0.85 이상

의미적 검색: 0.7 이상

문서 분류: 0.6 이상

추천 시스템: 0.5 이상

이러한 방법들을 조합하여 사용하면 다양한 상황에서 효과적인 유사도 검사를 수행할 수 있습니다. 특히 차원 조정을 통해 성능과 비용의 균형을 맞추는 것도 중요합니다.